In [16]:
# Imports needed for this ipython file

from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
import pickle
import os
from TwitterAPI import TwitterAPI

%matplotlib inline

In [17]:
def get_twitter(config_file):
    """ 
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [18]:
def robust_request(twitter, resource, params, max_tries=5):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        try:
            request = twitter.request(resource, params)
        except:
            print >> sys.stderr, 'Got Connection error: sleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)
            twitter = get_twitter('twitter.cfg')
            request = twitter.request(resource, params)
        if request.status_code == 200:
            return request
        elif "Not authorized" in request.text:
            return -1
        elif "page does not exist" in request.text:
            return -2
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [19]:
def get_followers(screen_name):
    """ Return a dictionary of the users that follows this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of dictionary, one per follower containing {index, user_info} pairs.
    Note: Many users follow more than 200 accounts; we will return those accounts too.
    """
    followers = {}
    cursor = -1
    key = 0
    rate_limit = 1
    while True:
        if rate_limit == 15:
            rate_limit = 1
            print >> sys.stderr, 'Avoided rate_limit error: \nsleeping for 15 minutes.'
            time.sleep(61 * 15)
        request = robust_request(twitter,'followers/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        json_response = request.json()
        
        # Get 200 followers from json_response
        for follower in json_response['users']:
                followers[follower['screen_name']] = follower
                key = key + 1
                
        # If user has < 200 followers or to retrieve last 200 followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 followers
        cursor = json_response["next_cursor"]
        rate_limit = rate_limit + 1
        print "Collected %d followers" % key
    return followers

In [20]:
def get_followers_and_pickle_object(account_name):
    """ Calls get_followers() to collect all followers of a Twitter account
        and then saves it by pickling for future processing purposes.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    followers = get_followers(account_name)
    fileObject = open("raw_data/" + account_name,'wb')
    pickle.dump(followers,fileObject)
    fileObject.close()
    return followers

In [34]:
def get_followers_from_pickle(account_name, path="raw_data/"):
    """ Returns all followers of a Twitter account by retrieving saved data,
        using pickle method.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
          path is a string representing the path of the pickled files
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    fileObject = open(path + account_name,'r')
    followers = pickle.load(fileObject)
    fileObject.close()
    return followers

In [ ]:
# Getting iit alumni association followers and pickling them
iit_alumni_followers = get_followers_and_pickle_object('iitalumniassn')

In [8]:
iit_alumni_followers = get_followers_from_pickle('iitalumniassn')
print "Found %d followers for iitalumniassn" %len(iit_alumni_followers.keys())

Found 1861 followers for iitalumniassn


In [6]:
# Getting iit career services followers and pickling them
iit_career_followers = get_followers_and_pickle_object('IITCareers')

In [9]:
iit_career_followers = get_followers_from_pickle('IITCareers')
print "Found %d followers for IITCareers" %len(iit_career_followers.keys())

Found 643 followers for IITCareers


In [ ]:
# Getting iit main university followers and pickling them
iit_univ_followers = get_followers_and_pickle_object('illinoistech')

In [10]:
iit_univ_followers = get_followers_from_pickle('illinoistech')
print "Found %d followers for IIT University" %len(iit_univ_followers.keys())

Found 5426 followers for IIT University


In [22]:
def get_all_screen_names(*args):
    """ Returns union of all followers of a particular university
    
    Args: Variables containing followers from different accounts in dictionary format.
    
    Returns : A set of intersection of keys from the dictionaries passed as input.
    """
    univ_followers = set()
    for followers in args:
        for user in followers.keys():
            univ_followers.add(user)
    return univ_followers

In [12]:
iit_followers = get_all_screen_names(iit_alumni_followers,iit_career_followers,iit_univ_followers)
print "Found %d unique accounts from %d accounts " %(len(iit_followers), 
            (len(iit_alumni_followers)+len(iit_career_followers)+len(iit_univ_followers)))

Found 6565 unique accounts from 7930 accounts 


In [13]:
# Pickling iit_followers for future calculations.
fileObject = open("processed_data/iit_followers",'wb')
pickle.dump(iit_followers,fileObject)
fileObject.close()

In [8]:
# Loading iit_followers from pickle.
fileObject = open("processed_data/iit_followers",'r')
iit_followers = pickle.load(fileObject)
fileObject.close()

In [23]:
def get_friends(screen_name):
    """ Return a dictionary of users that are friends of this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of {screen_name : description} per friend of the given account.
    """
    friends = {}
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'friends/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        
        if request == None:
            return 0
        
        if request == -1 :    # Authentication error : Can't access follower data
            return -1
        elif request == -2:   # page does not exist error
            return -2
        
        json_response = request.json()
        
        # Get 200 friends from json_response
        for friend in json_response['users']:
                #friends.append(friend['screen_name']) -- to_remove
                friends[friend['screen_name']] = friend['description']
                key = key + 1
                
        # If user has < 200 friends or to retrieve last friends followers
        if json_response["next_cursor"] == 0:
            break
        
        # Update cursor to get next 200 friends
        cursor = json_response["next_cursor"]
        print "Collected %d friends for %s" % (key,screen_name)
    print "Returning --- %d friends for %s" % (key,screen_name)
    return friends

In [ ]:
friends_of_iit_followers = { x : get_friends(x) for x in iit_followers}

In [24]:
def get_friends_and_pickle_object(univ_followers, univ_name):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: A list of strings representing the screen_names of all followers of a university,
          A string representing the university's name.
          
    Returns : A dictionary of { String , [List] } pairs,
              where String is a university follower's screen_name
              and List is a list of screen_names of their friends
    """
    friends_of_univ_followers = {}
    for follower in univ_followers:
        friends = get_friends(follower)
        friends_of_univ_followers[follower] = friends
        fileObject = open("raw_data/friends_of_" + univ_name + "_followers",'wb')
        pickle.dump(friends_of_univ_followers,fileObject)
        fileObject.close()
    return friends_of_univ_followers

In [ ]:
get_friends_and_pickle_object(iit_followers,"iit")

In [16]:
for idx,follower in enumerate(iit_followers):
    if "nancyzhang2" in follower:
        print idx

4717


In [21]:
iit_followers_2 =  list(iit_followers)[1548:]

In [25]:
#Need to be executed for second half split_pickle
def get_friend_ids(screen_name):
    """ Return a dictionary of users that are friends of this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A list of ids per friend of the given account.
    """
    friends = []
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'friends/ids', 
                                 {'screen_name': screen_name, 'count':5000, 'cursor': cursor})
        
        if request == None:
            return 0
        elif request == -1 :    # Authentication error : Can't access follower data
            return -1
        elif request == -2:   # Page does not exist error
            return -2
        
        json_response = request.json()
        
        # Get 5000 friends from json_response
        for friend_id in json_response['ids']:
            friends.append(friend_id)
        key = key + len(friends)
        
        # If user has < 200 friends or to retrieve last friends followers
        if json_response["next_cursor"] == 0:
            break
        
        # if user has > 50,000 friends, returning error code : -3
        if key > 50000 - 1:
            print "Found more than 50,000 friends. Returning error code = -3"
            return -3
            
        # Update cursor to get next 200 friends
        cursor = json_response["next_cursor"]
        print "Collected %d friends for %s" % (key,screen_name)
    print "Returning --- %d friends for %s" % (key,screen_name)
    return friends

In [11]:
#Need to be executed for second half split_pickle
def get_friend_ids_and_pickle_object(univ_followers, univ_name, split_pickle = False):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: A list of strings representing the screen_names of all followers of a university,
          A string representing the university's name.
          A boolean indicating whether to split the pickling method for each follower.
          
    Returns : A dictionary of { String , [List] } pairs,
              where String is a university follower's screen_name
              and List is a list of screen_names of their friends
    """
    friends_of_univ_followers = {}
    for follower in univ_followers:
        friends = get_friend_ids(follower)
        friends_of_univ_followers[follower] = friends
        
        if split_pickle == True:
            fileObject = open("raw_data/friend_ids_of_" + univ_name + "_" + follower
                              + "_followers",'wb')
        else:
            fileObject = open("raw_data/friend_ids_of_" + univ_name + "_followers",'wb')
        pickle.dump(friends_of_univ_followers,fileObject)
        fileObject.close()
        
        if split_pickle == True:
            friends_of_univ_followers.clear()
        
    return friends_of_univ_followers

In [ ]:
get_friend_ids_and_pickle_object(iit_followers_2,"iit2")

In [11]:
iit_followers_3 =  list(iit_followers)[2777:]
print len(iit_followers_3)

3788


In [ ]:
get_friend_ids_and_pickle_object(iit_followers_3,"iit3")

In [17]:
iit_followers_4 =  list(iit_followers)[4717:]

In [ ]:
get_friend_ids_and_pickle_object(iit_followers_4,"iit4")

#Collecting data for North Western University

In [26]:
def get_followers_and_robust_pickle(screen_name):
    """ 
    Retrieves and pickles a dictionary of dictionary, 
        one per follower containing { follower_screen_name: {follower_object} } pairs.
        Pickles by splitting to a different file every 1000 followers
        
    Args:
        screen_name: a string of a Twitter screen name
           
    Note: Many users follow more than 200 accounts; we will return those accounts too.
    """
    followers = {}
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'followers/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        json_response = request.json()
        
        # Get 200 followers from json_response
        for follower in json_response['users']:
                followers[follower['screen_name']] = follower
                key = key + 1
                
        # If user has < 200 followers or to retrieve last 200 followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 followers
        cursor = json_response["next_cursor"]
        
        # Pickling followers after 1000 followers and clearing followers
        if key % 1000 == 0:
            fileObject = open("raw_data/" + screen_name + str(key),'wb')
            pickle.dump(followers,fileObject)
            fileObject.close()
            followers = {}
            print "Pickling %d followers and clearing followers object" % key
        
        print "Collected %d followers" % key
    
    # Checking if followers is not empty
    if bool(followers) != False:
        fileObject = open("raw_data/" + screen_name + str(key+1),'wb')
        pickle.dump(followers,fileObject)
        fileObject.close()
        followers = {}
        print "Pickling %d followers and clearing followers object" % key

In [ ]:
# Getting North Western University Career services followers and pickling them
NUCareerAdvance_followers = get_followers_and_robust_pickle('NUCareerAdvance')

In [ ]:
# Getting North Western University Alumni followers and pickling them
nualumni_followers = get_followers_and_robust_pickle('nualumni')

In [ ]:
# Getting North Western University followers and pickling them
NorthwesternU_followers = get_followers_and_robust_pickle('NorthwesternU')

In [ ]:
def get_filenames(account_name, path = 'raw_data'):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: Path; which represents the sub directory to search for.
          Account_name; which represents the file names to be searched for against an account' screen_name.
          
    Returns : A list of filenames associated with an account
    """
    files = os.listdir(path)
    file_names = []
    for file_name in files:
        if account_name in file_name:
            file_names.append(file_name)
    return file_names

print get_filenames('friend_ids_of_nu',path="raw_data/nu_data/")

In [44]:
def aggregate_data(account_name,path = "raw_data/"):
    """
    Collect data from different pickled objects related to an account
    and aggregate them to another pickle
    
    Args: account_name; a string representing the screen_name of the account which we want to aggregate.
    """
    aggregated_followers = {}
    files = get_filenames(account_name,path)
    for file_name in files:
        old_followers = get_followers_from_pickle(file_name,path)
        aggregated_followers.update(old_followers)
    
    fileObject = open(path + account_name,'wb')
    pickle.dump(aggregated_followers,fileObject)
    fileObject.close()

In [30]:
aggregate_data('NorthwesternU')

In [22]:
aggregate_data('friend_ids_of_iit')

In [ ]:
aggregate_data('friend_ids_of_nu',path="raw_data/nu_data/")

In [31]:
aggregate_data('nualumni')

In [32]:
aggregate_data('NUCareerAdvance')

In [33]:
nu_univ_followers = get_followers_from_pickle('NorthwesternU')
print "Found %d followers for Northwestern University" %len(nu_univ_followers.keys())

Found 31999 followers for Northwestern University


In [34]:
nu_alumni_followers = get_followers_from_pickle('nualumni')
print "Found %d followers for Northwestern University" %len(nu_alumni_followers.keys())

Found 7816 followers for Northwestern University


In [35]:
nu_career_followers = get_followers_from_pickle('NUCareerAdvance')
print "Found %d followers for Northwestern University" %len(nu_career_followers.keys())

Found 3738 followers for Northwestern University


In [49]:
nu_followers = sorted(get_all_screen_names(nu_univ_followers,nu_alumni_followers,nu_career_followers))
print "Found %d unique accounts from %d accounts " %(len(nu_followers), 
            (len(nu_univ_followers)+len(nu_alumni_followers)+len(nu_career_followers)))

Found 35984 unique accounts from 43553 accounts 


In [52]:
# Pickling nu_followers as a SORTED LIST for future calculations.
fileObject = open("processed_data/nu_followers",'wb')
pickle.dump(nu_followers,fileObject)
fileObject.close()

In [14]:
# Loading nu_followers from pickle.
fileObject = open("processed_data/nu_followers",'r')
nu_followers = pickle.load(fileObject)
fileObject.close()

### Splitting university followers into two lists. To reduce time in collecting future data ( their friend ids ).

In [15]:
# Pickling first half of nu_followers for collecting their friend ids.
fileObject = open("processed_data/nu_followers_first_half",'wb')
pickle.dump(nu_followers[:len(nu_followers)/2],fileObject)
fileObject.close()

In [16]:
# Pickling second half of nu_followers for collecting their friend ids.
fileObject = open("processed_data/nu_followers_second_half",'wb')
pickle.dump(nu_followers[len(nu_followers)/2:],fileObject)
fileObject.close()

In [17]:
# Loading first half of nu_followers from pickle.
fileObject = open("processed_data/nu_followers_first_half",'r')
nu_followers_1 = pickle.load(fileObject)
fileObject.close()

In [18]:
# Loading second half of nu_followers from pickle.
fileObject = open("processed_data/nu_followers_second_half",'r')
nu_followers_2 = pickle.load(fileObject)
fileObject.close()

In [ ]:
# Collecting first half of nu_followers's friends' ids and pickling them separately
get_friend_ids_and_pickle_object(nu_followers_1, 'nu', split_pickle = True)

In [ ]:
# Collecting second half of nu_followers's friends' ids and pickling them separately
get_friend_ids_and_pickle_object(nu_followers_2, 'nu', split_pickle = True)

In [17]:
# Collecting all nu_followers's friends' ids and pickling them separately
get_friend_ids_and_pickle_object(nu_followers, 'nu', split_pickle = True)

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


Returning --- 1295 friends for 005ia
Returning --- 349 friends for 007MAV
Returning --- 882 friends for 007kengom
Returning --- 1964 friends for 0118mag3
Returning --- 20 friends for 0179849769
Returning --- 358 friends for 027_rmm
Returning --- 13 friends for 0356614b71b34e8
Returning --- 58 friends for 0599ab6909434a6
Returning --- 470 friends for 06122234
Returning --- 45 friends for 07_lauras
Returning --- 549 friends for 0838621955
Returning --- 599 friends for 08Warsono

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 158 friends for 0g_shemma
Returning --- 2007 friends for 1000Khalid
Returning --- 2098 friends for 1000WattAlexa
Returning --- 49 friends for 100101010100100
Returning --- 2004 friends for 1001chicago
Returning --- 151 friends for 100PERCENTHUTCH
Returning --- 946 friends for 100acreCo
Returning --- 832 friends for 100indecisions
Returning --- 39 friends for 100noob1
Returning --- 1913 friends for 101grads
Returning --- 10 friends for 1090779968

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


...


Returning --- 40 friends for AlishaAlishag
Returning --- 444 friends for AlisonDoerfler
Collected 5000 friends for AlisonDoyle
Collected 15000 friends for AlisonDoyle
Returning --- 28652 friends for AlisonDoyle
Returning --- 49 friends for AlisonLMayPhD
Returning --- 332 friends for AlisonNewton16
Returning --- 652 friends for Alison_W_Smith
Returning --- 1389 friends for AlissaDosSantos
Returning --- 124 friends for AlissaSchapiro
Returning --- 390 friends for AlissaZhu
Returning --- 502 friends for AliveintheFire
Returning --- 3 friends for AlixKramerMedil
Returning --- 235 friends for AlkhaniBaraa
Returning --- 348 friends for AllIWantFilm

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 249 friends for AllLawSchools
Returning --- 22 friends for AllThrillGill
Returning --- 59 friends for AllanPeck1
Returning --- 626 friends for AllanPlatt
Collected 5000 friends for AllegisGlobal
Returning --- 11257 friends for AllegisGlobal
Returning --- 372 friends for AlliGoetz
Returning --- 9 friends for AlliJobin
Returning --- 423 friends for AlliManus
Returning --- 1277 friends for AllianceChicago
Returning --- 77 friends for AllieGatorSmith
Returning --- 222 friends for AllieJennaaa
Returning --- 633 friends for AllieRasmus

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


TwitterConnectionError: ('Connection aborted.', gaierror(11004, 'getaddrinfo failed'))

In [19]:
for idx,follower in enumerate(nu_followers):
    if "AllieRasmus" in follower:
        print idx

1017


In [30]:
nu_followers.sort()
nu_followers_2 =  nu_followers[1017:]

In [ ]:
# Collecting all nu_followers's friends' ids and pickling them separately
get_friend_ids_and_pickle_object(nu_followers_2, 'nu', split_pickle = True)

Returning --- 633 friends for AllieRasmus
Returning --- 161 friends for AllieRubentweet
Returning --- 399 friends for AlliedSLS
Returning --- 7 friends for AllisonBCanade
Returning --- 38 friends for AllisonBMueller
Returning --- 26 friends for AllisonChen0925
Returning --- 61 friends for AllisonDavisUR
Returning --- 783 friends for AllisonETurner
Returning --- 1966 friends for AllisonLWong
Returning --- 11 friends for AllisonNaval
Returning --- 126 friends for Allison_Ledwon
Returning --- 300 friends for AllofE_HE

 Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 668 friends for AllstateCareers
Returning --- 27 friends for AlluringBeauty4
Returning --- 726 friends for AllyD2030
Returning --- 706 friends for Allys_Law
Returning --- 36 friends for AllysonWestling
Returning --- 1622 friends for AlmandoCoutinho
Returning --- 286 friends for AlohaPrintGroup
Returning --- 43 friends for AlonRozenshein
Returning --- 413 friends for AlonsoGdelCampo
Returning --- 128 friends for Alouettes1234
Returning --- 149 friends for AltCurr
Returning --- 40 friends for AltKatieDell
Returning --- 830 friends for Altair_US

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 41 friends for AlterBrewing
Returning --- 1752 friends for AltheaMcIntyre
Returning --- 1593 friends for AltoChicago
Returning --- 47 friends for Alto_Love49
Returning --- 1926 friends for AlumSPC
Returning --- 702 friends for AlumTalks
Returning --- 1985 friends for AlumVest
Returning --- 74 friends for AlumbunguFelix
Returning --- 1709 friends for AlumnForce
Returning --- 1025 friends for Alumni333
Returning --- 203 friends for AlumniApp
Returning --- 848 friends for AlumniBattle
Returning --- 349 friends for AlumniBridge
Returning --- 192 friends for AlumniChallenge

...

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 550 friends for COED_Recycling
Returning --- 20 friends for COLLEGEiNSIDERS
Returning --- 124 friends for CONECTAgroup
Returning --- 11 friends for COPE_NU
Returning --- 1757 friends for CORE_Institute
Returning --- 569 friends for COSseaton
Returning --- 288 friends for CPExperts
Returning --- 671 friends for CPFinn
Returning --- 427 friends for CPHSguidance
Returning --- 1683 friends for CPPEducation
Returning --- 120 friends for CPPaoloRecruits
Returning --- 116 friends for CPRGI
Returning --- 1178 friends for CPS_facts
Returning --- 885 friends for CPWeiss

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 23 friends for CPYMD
Returning --- 279 friends for CPjaxairport
Returning --- 1975 friends for CPracticesInc
Returning --- 438 friends for CRASHSCIENCE
Returning --- 309 friends for CRCWhat
Returning --- 1957 friends for CRCabinetry
Returning --- 91 friends for CRD_Recruit
Returning --- 553 friends for CRFChicago
Returning --- 133 friends for CRHasbrouck
Returning --- 10 friends for CRI_KA
Returning --- 2003 friends for CRLamberthHOK
Returning --- 1243 friends for CROSSMARKJobs
Returning --- 88 friends for CR__Brian
Returning --- 46 friends for CRafacalder11

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


In [8]:
for idx,follower in enumerate(nu_followers):
    if "CRafacalder11" in follower:
        print idx

2782


In [9]:
nu_followers_3 =  nu_followers[2782:]

In [ ]:
# Collecting all nu_followers's friends' ids and pickling them separately
get_friend_ids_and_pickle_object(nu_followers_3, 'nu', split_pickle = True)

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


Returning --- 46 friends for CRafacalder11
Returning --- 87 friends for CRuffstein
Returning --- 131 friends for CSBwildcats
Returning --- 1088 friends for CSConCampus
Returning --- 1979 friends for CSForJesus
Returning --- 1031 friends for CSFunding
Returning --- 78 friends for CSGICareers
Returning --- 27 friends for CSIProgramming
Returning --- 1144 friends for CSI_NSP
Returning --- 638 friends for CSKnellinger
Returning --- 224 friends for CSLawrence
Returning --- 1286 friends for CSM_Omaha
Returning --- 914 friends for CSNTF
Returning --- 915 friends for CSOResearch

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 1326 friends for CSUCareerCenter
Returning --- 314 friends for CSUSBCareers
Returning --- 1137 friends for CSUSBPreMed
Returning --- 399 friends for CSUSB_HPAC
Returning --- 448 friends for CSU_CTRE
Returning --- 1030 friends for CSUtoBigTen
Returning --- 55 friends for CSatFU
Returning --- 31 friends for CSdAnjou
Returning --- 230 friends for CShah85
Returning --- 36 friends for CShenkman
Returning --- 202 friends for CSherman5
Returning --- 222 friends for CShihabuddin
Returning --- 45 friends for CSteelberg

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


...

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 1873 friends for ChristianMurr17
Returning --- 177 friends for ChristianaS25
Returning --- 983 friends for ChristiePete77
Returning --- 121 friends for ChristieSohn
Returning --- 677 friends for Christie_Ileto
Returning --- 44 friends for ChristinDWarner
Returning --- 871 friends for Christina9473
Returning --- 112 friends for ChristinaEstell
Returning --- 244 friends for ChristinaHKim
Returning --- 123 friends for ChristinaStopka
Returning --- 222 friends for Christinafredr4

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 421 friends for ChristineVanDW
Returning --- 603 friends for Christine_McF
Returning --- 69 friends for Christine__CC
Returning --- 731 friends for ChristofToft
Returning --- 712 friends for ChristopherAyan
Returning --- 177 friends for ChristopherStat
Returning --- 300 friends for ChristyLeachSU
Returning --- 198 friends for ChristyLu0111
Returning --- 86 friends for Christy_N7
Returning --- 125 friends for Christy_Serrano
Returning --- 1455 friends for ChristynProFo
Returning --- 210 friends for Chryssa01

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


In [11]:
print len(nu_followers_3)

33202


In [19]:
for idx,follower in enumerate(nu_followers):
    if "Chryssa01" in follower:
        print idx

3542


In [21]:
nu_followers_4 = nu_followers[3542:len(nu_followers)/2]

In [23]:
print nu_followers_4[0]

Chryssa01


In [ ]:
# Collecting all nu_followers's friends' ids and pickling them separately
get_friend_ids_and_pickle_object(nu_followers_4, 'nu', split_pickle = True)

Returning --- 210 friends for Chryssa01
Returning --- 33 friends for ChuckBartling
Returning --- 634 friends for ChuckPorcelli
Returning --- 109 friends for Chujie_Chen
Returning --- 431 friends for Chunds19
Returning --- 29 friends for ChuyueYang
Returning --- 756 friends for CiMobileApp
Returning --- 1161 friends for CianBannon
Returning --- 922 friends for Cianoff
Returning --- 856 friends for CiaoBelu
Returning --- 381 friends for CiaranJDoyle
Returning --- 1896 friends for CicadaStore
Returning --- 24 friends for CierraLevy
Returning --- 525 friends for CierraMarie97
Returning --- 257 friends for CincinnatiMSTP

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 97 friends for CindyChang001
Returning --- 79 friends for CindyFiring
Returning --- 68 friends for CindyGalvin1
Returning --- 16 friends for CindyJOsman
Returning --- 100 friends for CindyLeeBrowne
Returning --- 147 friends for CinemaNero
Returning --- 40 friends for CineyJohn
Returning --- 1025 friends for Cinniebunnie
Returning --- 425 friends for Cinternships
Returning --- 122 friends for Cisca13
Returning --- 2024 friends for Ciscogiii

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 179 friends for CitadelCareers
Returning --- 42 friends for CitizenTy
Returning --- 214 friends for CitopatologiaG
Returning --- 1126 friends for CitrixCareers
Returning --- 187 friends for CityGirlLost
Returning --- 810 friends for CityLitTheater
Collected 5000 friends for CityWineryCHI
Returning --- 10140 friends for CityWineryCHI
Returning --- 1657 friends for CityYearMidwest
Returning --- 804 friends for CityofEvanston
Returning --- 175 friends for ClaireFontenot
Returning --- 195 friends for ClaireHaws

...

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 50 friends for LuvsPugs100
Returning --- 1977 friends for Lux_Roses
Returning --- 196 friends for Luxottica_Jobs
Returning --- 1920 friends for LuzyLed
Returning --- 1069 friends for Lwood847
Returning --- 177 friends for LycaHuang0621
Returning --- 741 friends for LyceumAgency
Returning --- 878 friends for LydiaBelanger
Returning --- 72 friends for LydiaChesnick
Returning --- 21 friends for LydiaGao1
Returning --- 713 friends for LydiaMHopper
Returning --- 543 friends for LyleEngineering
Returning --- 12 friends for LynGi_Z
Returning --- 1199 friends for Lyndab08

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 57 friends for LyndonSapozhnik
Returning --- 1029 friends for LyndseyMcK
Returning --- 311 friends for LyndseyMcKay
Returning --- 265 friends for LynnDolynchuk
Returning --- 9 friends for LynnHopton
Returning --- 85 friends for LynneQu
Returning --- 152 friends for LynnettePierre
Returning --- 274 friends for Lynnp69
Returning --- 931 friends for Lynseybt10
Returning --- 777 friends for LyricOpera
Returning --- 4714 friends for LyssaCurran
Returning --- 86 friends for LyssaWeave

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


In [12]:
temp_error_accounts = [u'usacolumbus69', u'gerenciatodos', u'pauljstill', u'Cascadia', 
                       u'CanyonDG', u'shopswindows', u'ranjan_srk', u'BerkeleyBrett',
                       u'LionsheadSocial', u'thejusticeconf', u'AdelmansCo', u'MikeBoehler', 
                       u'Brodalumab', u'roflgoo', u'SportingKC', u'ccrealty2', u'davidhoffmanca', 
                       u'BUBLIVE', u'NealSchaffer', u'TeeGreek']
get_friend_ids_and_pickle_object(temp_error_accounts, "iit5")

Collected 5000 friends for usacolumbus69
Returning --- 10002 friends for usacolumbus69
Collected 5000 friends for gerenciatodos
Collected 15000 friends for gerenciatodos
Collected 30000 friends for gerenciatodos
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for pauljstill
Collected 15000 friends for pauljstill
Returning --- 26089 friends for pauljstill
Collected 5000 friends for Cascadia
Collected 15000 friends for Cascadia
Returning --- 25489 friends for Cascadia
Collected 5000 friends for CanyonDG
Returning --- 10675 friends for CanyonDG
Collected 5000 friends for shopswindows

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Collected 15000 friends for shopswindows
Collected 30000 friends for shopswindows
Found more than 50,000 friends. Returning error code = -3
Returning --- 139 friends for ranjan_srk
Collected 5000 friends for BerkeleyBrett
Collected 15000 friends for BerkeleyBrett
Collected 30000 friends for BerkeleyBrett
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for LionsheadSocial
Collected 15000 friends for LionsheadSocial
Collected 30000 friends for LionsheadSocial
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for thejusticeconf
Returning --- 13524 friends for thejusticeconf
Collected 5000 friends for AdelmansCo

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Collected 15000 friends for AdelmansCo
Returning --- 25241 friends for AdelmansCo
Collected 5000 friends for MikeBoehler
Collected 15000 friends for MikeBoehler
Collected 30000 friends for MikeBoehler
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for Brodalumab
Collected 15000 friends for Brodalumab
Collected 30000 friends for Brodalumab
Found more than 50,000 friends. Returning error code = -3
Returning --- 4393 friends for roflgoo
Collected 5000 friends for SportingKC
Returning --- 10494 friends for SportingKC
Collected 5000 friends for ccrealty2
Collected 15000 friends for ccrealty2

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Collected 30000 friends for ccrealty2
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for davidhoffmanca
Collected 15000 friends for davidhoffmanca
Collected 30000 friends for davidhoffmanca
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for BUBLIVE
Collected 15000 friends for BUBLIVE
Collected 30000 friends for BUBLIVE
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for NealSchaffer
Collected 15000 friends for NealSchaffer
Collected 30000 friends for NealSchaffer
Found more than 50,000 friends. Returning error code = -3
Collected 5000 friends for TeeGreek

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Collected 15000 friends for TeeGreek
Collected 30000 friends for TeeGreek
Found more than 50,000 friends. Returning error code = -3


{u'AdelmansCo': [82192102,
  505440302,
  2432548092L,
  143528652,
  129882996,
  151337285,
  43084095,
  872730668,
  95083299,
  177197452,
  188532686,
  163105866,
  184184224,
  184203060,
  229509122,
  223532414,
  211632786,
  229549784,
  205302425,
  247935695,
  183143642,
  267467391,
  142698446,
  390969254,
  298855671,
  342985243,
  245113074,
  109251482,
  136249277,
  127324156,
  114875943,
  615344550,
  569353902,
  624943260,
  1004906010,
  151476458,
  549392161,
  584134945,
  180660817,
  510489112,
  717397880,
  397335122,
  872991938,
  1420740331,
  1406131026,
  1242353329,
  1412366977,
  1161576870,
  1224900890,
  1472322865,
  243483837,
  1488947678,
  1154541175,
  1542069564,
  163660242,
  1623174458,
  313712160,
  1609264572,
  1045695522,
  44145131,
  141563014,
  180655149,
  155290700,
  101258547,
  1947709099,
  89860239,
  486303487,
  17926194,
  138152191,
  1418470400,
  376359822,
  352653786,
  67416037,
  131190306,
  2411582911

In [24]:
iit_friend_ids = get_followers_from_pickle("friend_ids_of_iit")

In [ ]:
print iit_friend_ids["usacolumbus69"]